In [ ]:
#CLIENT PROGRAM
#CHANGE IT TO CLASS... MAKE GLOBAL VARIABLES AS CLASS VARIABLES AND FUNCTIONS AS CLASS METHODS
#LEARN OOPS CONCEPT
import os
import socket
import time
HEADER = 64
PORT = 5050
FORMAT = 'utf-8'
DISCONNECT_MESSAGE = "!DISCONNECT"
SERVER = "192.168.1.11"
ADDR = (SERVER, PORT)
client_ack = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #HANDSHAKE TCP
client_ack.connect(ADDR)
client_dt = socket.socket(socket.AF_INET, socket.SOCK_STREAM) #DATA TRANSFER TCP
client_dt.connect(ADDR)

def encoding(msg):
    message = msg.encode(FORMAT)
    msg_length = len(message)
    send_length = str(msg_length).encode(FORMAT)
    send_length += b' ' * (HEADER - len(send_length))
    return message,send_length

def handshake(sourcefile):
    msg,length = encoding(sourcefile)
    client_ack.send(length)
    client_ack.send(msg)
    file_ack = client_ack.recv(1024).decode(FORMAT)
    filename,size = file_ack.split(',')
    print(f"{filename} with size {size} Found on SERVER! ")
    return int(size),filename
    
def disconnect(msg,client):
    msg,length = encoding(msg)
    client.send(length)
    client.send(msg)
    
def receiving(target,source,location,size):
    msg,length = encoding(f"{source},0")
    client_dt.send(length)
    client_dt.send(msg)
    tick = time.time()
    print("Started receiving")
    Receiving = True
    os.chdir(location)
    count = 0
    recvd_size = 0
    size2 = size
    data = ""
    while True:
        with open(target,'ab') as dest:
            data = ""
            if size2 == 0:
                break
            data = client_dt.recv(2048000)
            recvd_size += len(data)
            if recvd_size >= 20480000:
                count += 1 
                print(f"Received {count*20} MB out of {size/1024000} MB")
                recvd_size = 0
            size2 -= len(data)
            dest.write(data)
    clock = time.time()
    print(f"Receiving complete in {(clock-tick):.2f} seconds...")
    print(os.path.getsize(target))
    disconnect(DISCONNECT_MESSAGE, client_dt)
    disconnect(DISCONNECT_MESSAGE, client_ack)

In [ ]:
#CALL FUNCTIONS
sourcefile = "Textbook.pdf"
sourcefile += ",1"
everything = "sathya.mkv,D:\\Shareable Folder"
target,location = everything.split(',')
size,source = handshake(sourcefile)
receiving(target,source,location,size)